# IBA Probe for Specific Hot/Cold Interface

In this tutorial we will demonstrate the "Specific Hot/Cold Interface" probe that is used to messure specific traffic load on select interfaces and detect when traffic load increases (hot) or decreses (cold) unexpectedly.

The first thing we need to do to is setup our AOS server and identify the blueprint we want to use.

In [1]:
aos_server = '172.20.91.3'
blueprint_name = "rack-based-blueprint-f03bfadb"

Next let's create a AOS python client that we will use to create our probe

In [2]:
import json
from aospy.swagger import Client

aos = Client('https://' + aos_server)

Now let's use that client to get the blueprint, so we can have the blueprint_id value we will need when we post the probe.

In [3]:
from aospy.swagger.groups.blueprints import Blueprints
bp_index = Blueprints(aos)
bp = bp_index[blueprint_name]

In [4]:
print bp.id

rack-based-blueprint-f03bfadb


Now let's import the hot/cold interface probe from the library

In [5]:
from aospy.ibaprobelib.specific_hot_cold_interface import specific_hotcold_ifcounter_probe

Next let's create the hot/cold interface probe payload information that we will need.  The probe takes a few parmateres which we can see if we introspect the function:

In [ ]:
?specific_hotcold_ifcounter_probe

The following paramaters are required:

`label` - Descriptive name for the probe   
`interfaces` - list of interfaces   
    - system_label - label of system node that hosts given interface  
    - interface_name - if_name of interface node representing given interface  
 
`if_counter` - Type of interface counter:  
    - tx_multicast_packets, rx_bytes, rx_error_packets, tx_error_packets,   
    - tx_unicast_packets, rx_discard_packets, rx_broadcast_packets,   
    - tx_broadcast_packets, rx_multicast_packets, tx_bytes, tx_discard_packets,   
    - rx_unicast_packets 
      
`min` - minimum level of counter  
`max` - maximum level of counter  
`max_hot_interface_percentage` - maximum percentage of hot interfaces on a device  
`max_cold_interface_percentage` - maximum percentage of cold interfaces on a device  
`average_period` - period over which to average input counter samples  
`duration` - number of seconds of recent-history over which interface counter   
hot/cold status will be considered  
`threshold_duration` - sum total of number of seconds of recent-history for which   
interface must be hot/cold for anomaly to be raised  
`anomaly_history_count` - number of anomaly flaps that will be recorded for inspection  


So for the case of reuse, we will define these variables here:

In [6]:
probe_label = 'DC1-hotcold-Spine1'
interfaces = [
            {'system_label':'spine1', 'interface_name':'Ethernet1'},
             {'system_label':'spine1', 'interface_name':'Ethernet2'}
             ]
if_counter = 'tx_unicast_packets'
min = 12500
max = 12500000
max_hot_interface_percentage = 5
max_cold_interface_percentage = 5
average_period = 30
duration = 60
threshold_duration = 120
anomaly_history_count = 500

Notice how `interface` is used to provide a list of interfaces for use with the probe.   
To fined the interfaces for each node you can look in the Links section of the topology

<img src="media/specific_hotcold2.png">

Now let's create the body data from these variables:

In [7]:
probe_body = specific_hotcold_ifcounter_probe(label=probe_label, interfaces=interfaces, if_counter=if_counter, min=min, max=max, max_hot_interface_percentage=max_hot_interface_percentage, max_cold_interface_percentage=max_cold_interface_percentage, average_period=average_period, duration=duration, threshold_duration=threshold_duration, anomaly_history_count=anomaly_history_count)

If you'd like to dump the contents of the probe_body, you can use the following:

In [8]:
# print json.dumps(probe_body, indent=2)

Finally, post the probe body to AOS to create the probe.

In [9]:
probes_url = bp.url + "/probes"

In [10]:
probes_url

u'https://172.20.91.3/api/blueprints/rack-based-blueprint-f03bfadb/probes'

In [11]:
resp = aos.remote.api.post(probes_url, json=probe_body)

In [12]:
resp.ok

True

Now we have our IBA Probe ID:

In [13]:
probe_id = resp.json()['id']

print probe_id

8d9d4635-bdf4-4320-9f0b-f8d1704a257d


And now we can see the IBA active probe in the UI:

<img src="media/hotcold_interface1.png">

And then select the probe on the UI to get the details:

<img src="media/specific_hotcold3.png">

# Delete the IBA Hot/Cold Interface Probe

If you want to remove the IBA probe, you can use the API to do so, like the following; just uncomment this out.

In [14]:
resp = aos.remote.api.delete(probes_url + "/%s" % probe_id)
resp.ok

True